In [ ]:
%load_ext autoreload
%autoreload 2

import pygame
import time
import numpy as np

from game import agent_by_mark, TicTacToeEnv, next_mark # possibly need to set export DISPLAY=:0 in terminal?
from play_human import HumanAgent
# does this need to run?
# pygame.display.list_modes()

## TO RUN VIDEOSTUFF ON WINDOWS / WSL:
- 1: start xming
- 2: in terminal launching notebook, run export DISPLAY=:0

In [ ]:

NUM_ROWS = 6
NUM_COLS = 7

CELL_RADIUS = 20
CELL_PADDING = 15

BOARD_HEIGHT = NUM_ROWS * (2*CELL_RADIUS + CELL_PADDING) + 2*CELL_PADDING
BOARD_WIDTH = NUM_COLS * (2*CELL_RADIUS + CELL_PADDING) + 2*CELL_PADDING

#VELOCITY = 1
MARKER_RADIUS = CELL_RADIUS

BOARD_COLOR = pygame.Color('green')
CELL_COLOR = pygame.Color('black')
MARKER_1_COLOR = pygame.Color('blue')
MARKER_2_COLOR = pygame.Color('red')

DROP_PANEL_WIDTH = BOARD_WIDTH
DROP_PANEL_HEIGHT = 80
DROP_PANEL_COLOR = pygame.Color('black')

SCREEN_WIDTH = BOARD_WIDTH
SCREEN_HEIGHT = BOARD_HEIGHT + DROP_PANEL_HEIGHT

In [ ]:
pygame.init()

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) # returns the surface where you can draw stuff
pygame.draw.rect(screen, DROP_PANEL_COLOR, pygame.Rect((0,0), (DROP_PANEL_WIDTH, DROP_PANEL_HEIGHT)))
pygame.draw.rect(screen, BOARD_COLOR, pygame.Rect((0,DROP_PANEL_HEIGHT), (SCREEN_WIDTH, SCREEN_HEIGHT)))
pygame.display.flip()


# draw empty cells

def draw_empty_board():
    global screen
    global CELL_RADIUS, CELL_PADDING, CELL_COLOR
    global NUM_ROWS, NUM_COLS
    
    IDXS_TO_PIXEL_CORDS_MAP = dict()

    y_pos = DROP_PANEL_HEIGHT + CELL_RADIUS + CELL_PADDING
    for row_idx in range(NUM_ROWS):
        x_pos = CELL_RADIUS + CELL_PADDING
        for col_idx in range(NUM_COLS):
            IDXS_TO_PIXEL_CORDS_MAP[(row_idx, col_idx)] = (x_pos, y_pos)
            pygame.draw.circle(screen, CELL_COLOR, (x_pos, y_pos), CELL_RADIUS)
            x_pos += 2 * CELL_RADIUS + CELL_PADDING
        y_pos += 2 * CELL_RADIUS + CELL_PADDING
    
    return IDXS_TO_PIXEL_CORDS_MAP

IDXS_TO_PIXEL_CORDS_MAP = draw_empty_board()


def draw_board(board):
    global IDXS_TO_PIXEL_CORDS_MAP
    color_map = {0: CELL_COLOR, 1: MARKER_1_COLOR, 2: MARKER_2_COLOR}
    
    if isinstance(board, tuple): # undo board flattening
        board = np.array(board).reshape(6, 7)
    
    for row_index in range(board.shape[0]):
        for col_index in range(board.shape[1]):
            marker_value = board[row_index, col_index]
            marker_color = color_map[int(marker_value)]
            pygame.draw.circle(screen, marker_color, IDXS_TO_PIXEL_CORDS_MAP[(row_index, col_index)], CELL_RADIUS)

# init backend
env = TicTacToeEnv(show_number=False)
agents = [HumanAgent('O'), HumanAgent('X')]
state = env.reset()
board, mark = state
done = False
draw_board(board)

# run game

pygame.display.flip()
while True:
    e = pygame.event.poll()
    if e.type == pygame.QUIT:
        break
        
    pygame.display.flip()
    
    #checks if a mouse is clicked
    if e.type == pygame.MOUSEBUTTONDOWN:
        
        if 0 <= mouse_y <= DROP_PANEL_HEIGHT: # "actionpanel" is clicked
            # todo: infer action type from mouseclick position

            CLICK_ACTION = int(mouse_x // (SCREEN_WIDTH/NUM_COLS)) # mocked --- should be derived from click-area
            agent = agent_by_mark(agents, next_mark(mark))
            env.show_turn(True, mark)
            ava_actions = env.available_actions()
            action, action_is_valid = agent.pygame_act(ava_actions,CLICK_ACTION)
            
            if not action_is_valid:
                print('Invalid action not supported, exiting!')
                pygame.quit()
                break
                
            state, reward, done, info = env.step(action)
            board, mark = state
            draw_board(board)
            if done:
                env.show_result(True, mark, reward)
                #break
             
    mouse_x, mouse_y = pygame.mouse.get_pos()


## OLD PONG STUFF

In [ ]:
class Ball:
    
    RADIUS = 20
    
    def __init__(self, x, y, vx, vy):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        
    def show(self, color):
        global screen # take screen from global vars        
        pygame.draw.circle(screen, color, (self.x, self.y), self.RADIUS)
    
    def update(self):
        global ballColor, bgColor
        self.show(bgColor) # hides the current ball
        newx = self.x + self.vx
        newy = self.y + self.vy
        
        if newx < BORDER + self.RADIUS:
            self.vx = -self.vx
        elif newy < BORDER + self.RADIUS or newy > SCREEN_HEIGHT - (BORDER + self.RADIUS):
            self.vy = -self.vy
        else:
            self.x = newx
            self.y = newy
            self.show(ballColor)
         
class Paddle:
    PADDLE_WIDTH = 20
    PADDÖE_HEIGHT = 100

    def __init__(self, y):
        self.y = y

    def show(self, color):
        global screen, paddleColor
        pygame.draw.rect(screen, paddleColor, )


In [ ]:
pygame.init()

SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 600
BORDER = 20
VELOCITY = 1

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) # returns the surface where you can draw stuff

fgColor = pygame.Color('green')
bgColor = pygame.Color('black')
ballColor = pygame.Color('red')
paddleColor = pygame.Color('yellow')

# draw the borders
pygame.draw.rect(screen, fgColor, pygame.Rect((0,0), (SCREEN_WIDTH,BORDER)))
pygame.draw.rect(screen, fgColor, pygame.Rect((0,0), (BORDER, SCREEN_HEIGHT)))
pygame.draw.rect(screen, fgColor, pygame.Rect((0, SCREEN_HEIGHT-BORDER), (SCREEN_WIDTH,BORDER)))

# draw the ball
ballplay = Ball(SCREEN_WIDTH-Ball.RADIUS, SCREEN_HEIGHT // 2, -VELOCITY, VELOCITY)
ballplay.show(ballColor)

pygame.display.flip()

time.sleep(3)

while True:
    e = pygame.event.poll()
    if e.type == pygame.QUIT:
        break
        #time.sleep(10)

    pygame.display.flip()
    ballplay.update()
        
pygame.quit()

In [ ]:
pygame.quit()

In [ ]:
help(pygame.draw.rect)

In [ ]:
import numpy 

Notes -- pygame / graphics with WSL

WSL stödjer inte grafiska saker (linux kör X11)

Dock finns något som heter Xming som gör så att X11 funkar inuti windows(?)
- https://stackoverflow.com/questions/53124539/pygame-video-not-initialized-and-no-video-found-using-ubuntu-terminal-from-wi

X-ming downloads: https://sourceforge.net/projects/xming/
